In [1]:
import socket

afqdn = socket.getfqdn() # your /etc/hosts should be set up to contain fully qualified domain name of the machine
afqdn = 'nnlicv431.inn.intel.com'
afqdn

'nnlicv431.inn.intel.com'

## 1. Make sure you can run the `fx` command.

## 2. Create a workspace for the new federation project.

In [2]:
!fx workspace create --prefix ~/workspace --template torch_cnn_histology
%cd ~/workspace

Intel FLedge - Secure Federated Learning at the Edge™                           

Creating Workspace Directories
Creating Workspace Certifications
Creating Workspace Templates
You are using pip version 18.1, however version 20.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

New workspace directory structure:
workspace
├── code
│   ├── keras_cnn.py
│   ├── __pycache__
│   │   ├── __init__.cpython-36.pyc
│   │   ├── pt_cnn.cpython-36.pyc
│   │   ├── histology_utils.cpython-36.pyc
│   │   └── pthistology_inmemory.cpython-36.pyc
│   ├── mnist_utils.py
│   ├── tfmnist_inmemory.py
│   ├── pthistology_inmemory.py
│   ├── __init__.py
│   ├── histology_utils.py
│   ├── requirements.txt
│   └── pt_cnn.py
├── agg_to_col_two_signed_cert.zip
├── requirements.export.txt
├── cert
│   ├── ca
│   │   ├── root-ca.csr
│   │   ├── signing-ca
│   │   ├── root-ca.crt
│   │   ├── signing-ca.crt
│   │   ├── root-ca
│   │   └── signing-ca.csr
│   ├── client
│   │   ├

## 3. Initialize the plan
Although it is possible to train models from scratch, it is assumed that in many cases the federation may perform fine-tuning of a previously-trained model. For this reason, the pre-trained weights for the model will be stored within protobuf files on the aggregator and passed to the collaborators during initialization. As seen in the YAML file, the protobuf file with the initial weights is expected to be found in the file `torch_cnn_mnist_init.pbuf`. Let's just create an initial set of random model weights and put it into that file by running the command:

In [3]:
!fx plan initialize -a $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

[18:31:48] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1607095908.4073274-746187;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1607095908.409751-89977;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section collaborator from file plan/defaults/collaborator.yaml.                                              ]8;id=1607095908.4125698-284876;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFA

                    network:                                                                                                                                    
                      defaults: plan/defaults/network.yaml                                                                                                      
                      settings:                                                                                                                                 
                        agg_addr: nnlicv431                                                                                                                     
                        agg_port: 56518                                                                                                                         
                        cert_folder: cert                                                                                                                       
                        disable_cl

PyTorchCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(160, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv7): Conv2d(672, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv8): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=95904, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=8, bias=True)
)
[18:31:54] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                        ]8;id=1607095914.0813074-677771;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-p

## 4. Run the Certificate Authority command.
This will setup the Aggregator node as the Certificate Authority for the Federation. All certificates will be signed by the aggregator. The command will create a simple database file to keep track of all issued certificates.

In [4]:
!fx workspace certify

Intel FLedge - Secure Federated Learning at the Edge™                           

Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request
1.4 Create CA Certificate
2.  Create Signing Certificate
2.1 Create Directories
2.2 Create Database
2.3 Create Signing Certificate CSR
2.4 Sign Signing Certificate CSR
3   Create Certificate Chain

Done.

 ✔️ OK


## 5. Run the aggregator certificate creation command.

In [5]:
!fx aggregator generate-cert-request --fqdn $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

Creating AGGREGATOR certificate key pair with following settings: CN=nnlicv431.inn.intel.com, SAN=DNS:nnlicv431.inn.intel.com
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


## 6. Run the aggregator certificate signing command

In [6]:
!fx aggregator certify -s --fqdn $afqdn

Intel FLedge - Secure Federated Learning at the Edge™                           

The CSR Hash for file agg_nnlicv431.inn.intel.com.csr =  2302b01bb32e2dee78208747aeff01c60b70db9e805da08f1f3944552594a478

 Signing AGGREGATOR certificate key pair
  Moving AGGREGATOR certificate key pair to: cert/server

 ✔️ OK


## 7. Export the workspace so that it can be imported to the collaborator nodes.

In [7]:
!fx workspace export

Intel FLedge - Secure Federated Learning at the Edge™                           

[18:32:01] INFO     FL-Plan hash is 7da62f28238a0d52ab0db363b2fe9a46                                                                                 ]8;id=1607095921.0994592-9118;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:204
           INFO     plan.yaml is already frozen                                                                                                       ]8;id=1607095921.1016436-451978;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:45
Writing torch==1.6.0
 to requirements.txt...
Writing torchvision==0.7.0
 to requirements.txt...
requirements.export.txt written.
Workspace exported to archive: workspace.zip

 ✔️ OK


## 14. Sign collaborators certificates

In [8]:
!fx collaborator certify -s --request-pkg col_one_to_agg_cert_request.zip

Intel FLedge - Secure Federated Learning at the Edge™                           

The CSR Hash for file col_one.csr =  152992fca165b3dde37842b2b2ad2a8ade4c50bffc7ffa6a6c24b70c69826a58

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering one in plan/cols.yaml

 ✔️ OK


In [9]:
!fx collaborator certify -s --request-pkg col_two_to_agg_cert_request.zip

Intel FLedge - Secure Federated Learning at the Edge™                           

The CSR Hash for file col_two.csr =  4ba2331d1e700201b889119dab2c48dcaf678c88cae6add38b52d13bfd1a4379

 Signing COLLABORATOR certificate key pair
  Moving COLLABORATOR certificate key pair to: cert/client

Registering two in plan/cols.yaml

 ✔️ OK


In [16]:
ls

agg_to_col_one_signed_cert.zip   __init__.py
agg_to_col_two_signed_cert.zip   logs/
cert/                            plan/
code/                            requirements.export.txt
col_one_to_agg_cert_request.zip  requirements.txt
col_two_to_agg_cert_request.zip  save/
data/                            workspace.zip


## Start the aggregator.

In [10]:
!fx aggregator start

Intel FLedge - Secure Federated Learning at the Edge™                           

[18:40:55] INFO     Loading DEFAULTS for section aggregator from file plan/defaults/aggregator.yaml.                                                  ]8;id=1607096455.9674318-401819;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section assigner from file plan/defaults/assigner.yaml.                                                      ]8;id=1607096455.9713798-767882;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DEFAULTS for section collaborator from file plan/defaults/collaborator.yaml.                                              ]8;id=1607096455.9760892-760982;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:94
           INFO     Loading DE

           INFO     Settings 🡆 {'task_groups': [{'name': 'train_and_validate', 'percentage': 1.0, 'tasks': ['aggregated_model_validation', 'train',  ]8;id=1607096456.0577903-936036;file:///home/itrushkin/venvs/fledge/lib/python3.6/site-packages/fledge/federated/plan/plan.py\plan.py]8;;\:167
                    'locally_tuned_model_validation']}], 'authorized_cols': ['one', 'two'], 'rounds_to_train': 20, 'tasks':                                     
                    {'aggregated_model_validation': {'function': 'validate', 'kwargs': {'apply': 'global', 'metrics': ['acc']}},                                
                    'locally_tuned_model_validation': {'function': 'validate', 'kwargs': {'apply': 'local', 'metrics': ['acc']}}, 'train':                      
                    {'function': 'train_batches', 'kwargs': {'metrics': ['loss']}}, 'defaults': 'plan/defaults/tasks_torch.yaml', 'settings': {}}}              
           INFO     Override 🡆 {'defaults': 'plan/defaults/a

At this point, the aggregator is running and waiting for the collaborators to connect. When all of the collaborators connect, the aggregator starts training. When the last round of training is complete, the aggregator stores the final weights